In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 46.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9a057b33fd174e64e129f18c3687f21186f986abda994ec70b6d41a2cce3b8e3
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

--2022-10-19 20:20:53--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274079476 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop2.tgz’

spark-3.3.0-bin-had 100%[===================>] 261.38M   220MB/s    in 1.2s    

2022-10-19 20:20:54 (220 MB/s) - ‘spark-3.3.0-bin-hadoop2.tgz’ saved [274079476/274079476]



In [4]:
!tar xf spark-3.3.0-bin-hadoop2.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop2"

In [6]:
!pip install petastorm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 285 kB 4.7 MB/s 
     |████████████████████████████████| 44 kB 1.2 MB/s 


In [7]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 4.8 MB/s 
     |████████████████████████████████| 578.0 MB 14 kB/s 
     |████████████████████████████████| 5.9 MB 45.2 MB/s 
     |████████████████████████████████| 438 kB 67.2 MB/s 
     |████████████████████████████████| 1.7 MB 53.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [9]:
import findspark
findspark.init()

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .config("spark.csci316.pandas.randomforest", "false") \
    .getOrCreate()
spark

In [15]:
#Read the csv file
data = spark.read.csv("/content/drive/MyDrive/CSCI316/cleanWithHeader.csv", inferSchema=True, header=True)

In [11]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import tensorflow_decision_forests as tfdf
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import math

/usr/local/lib/python3.7/dist-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [16]:
data.show(10)

+-------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+-----------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|revol_util_imputed|last_pymnt

In [17]:
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- annual_inc: integer (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- out_prncp: double (nullable = true)
 |-- out_prncp_inv: double (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- total_pymnt_inv: double (nullable = true)
 |-- total_rec_prncp: double (nullable = true)
 |-- total_rec_int: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- recoveries: double (nullable = true)
 |-- col

In [18]:
df_train, df_val = data.randomSplit([0.7, 0.3], seed=135)

df_train = df_train.repartition(2)
df_val = df_val.repartition(2)

In [19]:
df_train.show()

+--------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+-----------+
|      id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|revol_util_imputed|last_pym

In [20]:
print(f"train: {df_train.count()}, val: {df_val.count()}")

train: 598745, val: 256998


**Convert from PySpark to TensorFlow**

In [21]:
print(f"train: {df_train.count()}, val: {df_val.count()}")

train: 598745, val: 256998


In [22]:
train_dataset = df_train.toPandas()

In [23]:
test_dataset = df_val.toPandas()

In [24]:
x_train = train_dataset.iloc[:,0:42]
y_train = train_dataset.iloc[:,-1]

In [25]:
train_dataset.default_ind.value_counts()

0    566164
1     32581
Name: default_ind, dtype: int64

In [26]:
x_test = test_dataset.iloc[:,0:42]
y_test = test_dataset.iloc[:,-1]

In [27]:
x_train.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,application_type_index,acc_now_delinq_index,grade_index,purpose_index,home_ownership_index,emp_length_index,verification_status_index,sub_grade_index,term_index,initial_list_status_index
0,36178144,38879659,29400,29400,29400.0,8.19,598.81,65000,9.29,0,...,0.0,0.0,2.0,1.0,0.0,10.0,0.0,9.0,1.0,1.0
1,7345138,9007271,16000,16000,16000.0,19.20,588.12,85000,13.17,0,...,0.0,0.0,3.0,1.0,2.0,0.0,1.0,14.0,0.0,0.0
2,2375049,2837232,35000,35000,35000.0,14.33,1201.84,200000,15.51,0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0
3,32119501,34722789,15000,15000,15000.0,11.67,331.18,55000,9.73,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
4,16742417,18854944,18000,18000,18000.0,13.35,412.79,80000,13.85,0,...,0.0,0.0,1.0,1.0,1.0,9.0,0.0,3.0,1.0,1.0


In [28]:
y_train.tail()

598740    0
598741    0
598742    0
598743    0
598744    0
Name: default_ind, dtype: int32

In [29]:
y_train.nunique()

2

In [30]:
tf_train = tfdf.keras.pd_dataframe_to_tf_dataset(train_dataset, label="default_ind")
tf_test = tfdf.keras.pd_dataframe_to_tf_dataset(test_dataset, label="default_ind")

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [31]:

print(tf_train.take(1))

<TakeDataset element_spec=({'id': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'member_id': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'loan_amnt': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'funded_amnt': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'funded_amnt_inv': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'int_rate': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'installment': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'annual_inc': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'dti': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'delinq_2yrs': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'inq_last_6mths': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'open_acc': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'pub_rec': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'revol_bal': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'total_acc': TensorSpec(s

In [32]:
type(tf_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [39]:
model_1 = tfdf.keras.RandomForestModel(verbose=2, num_trees = 1, max_depth=500)

# Train the model.
model_1.fit(tf_train)

Use 2 thread(s) for training
Use /tmp/tmp4hwtmmde as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'id': <tf.Tensor 'data_12:0' shape=(None,) dtype=int32>, 'member_id': <tf.Tensor 'data_19:0' shape=(None,) dtype=int32>, 'loan_amnt': <tf.Tensor 'data_18:0' shape=(None,) dtype=int32>, 'funded_amnt': <tf.Tensor 'data_8:0' shape=(None,) dtype=int32>, 'funded_amnt_inv': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'int_rate': <tf.Tensor 'data_16:0' shape=(None,) dtype=float64>, 'installment': <tf.Tensor 'data_15:0' shape=(None,) dtype=float64>, 'annual_inc': <tf.Tensor 'data_1:0' shape=(None,) dtype=int32>, 'dti': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'delinq_2yrs': <tf.Tensor 'data_5:0' shape=(None,) dtype=int32>, 'inq_last_6mths': <tf.Tensor 'data_14:0' shape=(None,) dtype=int32>, 'open_acc': <tf.Tensor 'data_20:0' shape=(None,) dtype=int32>, 'pub_rec': <tf.Tensor 'data_24:0' shape=(None,) dtype=int32>, 'revol_bal': <tf.

[INFO kernel.cc:813] Start Yggdrasil model training
[INFO kernel.cc:814] Collect training examples
[INFO kernel.cc:422] Number of batches: 599
[INFO kernel.cc:423] Number of examples: 598745
[INFO kernel.cc:836] Training dataset:
Number of records: 598745
Number of columns: 43

Number of columns by type:
	NUMERICAL: 42 (97.6744%)
	CATEGORICAL: 1 (2.32558%)

Columns:

NUMERICAL: 42 (97.6744%)
	0: "acc_now_delinq_index" NUMERICAL mean:0.00495035 min:0 max:9 sd:0.0775336
	1: "annual_inc" NUMERICAL mean:75027.9 min:0 max:9e+06 sd:63925.1
	2: "application_type_index" NUMERICAL mean:0.000537792 min:0 max:1 sd:0.0231841
	3: "collection_recovery_fee" NUMERICAL mean:4.9161 min:0 max:5694.09 sd:61.232
	4: "collections_12_mths_ex_med_imputed" NUMERICAL mean:0.0142765 min:0 max:16 sd:0.133118
	5: "delinq_2yrs" NUMERICAL mean:0.311861 min:0 max:39 sd:0.857336
	6: "dti" NUMERICAL mean:18.119 min:0 max:9999 sd:15.4077
	7: "emp_length_index" NUMERICAL mean:3.5233 min:0 max:11 sd:3.56217
	8: "funded_am

Model trained in 0:00:05.517713
Compiling model...
Model compiled.


In [40]:
model_1.compile(metrics=["accuracy"])

In [41]:
evaluation = model_1.evaluate(tf_test)

257/257 [==============================] - 10s 39ms/step - loss: 0.0000e+00 - accuracy: 0.9971


In [42]:
print(evaluation)

[0.0, 0.9971400499343872]


In [43]:
model_1.make_inspector().evaluation()

Evaluation(num_examples=220564, accuracy=0.9972524981411292, loss=0.09902999971877184, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)

In [44]:
model_1.make_inspector().variable_importances()

{'SUM_SCORE': [("recoveries" (1; #27), 55415.62644768506),
  ("total_rec_prncp" (1; #39), 18530.06597763265),
  ("out_prncp_inv" (1; #22), 16668.339879446088),
  ("last_pymnt_amnt_imputed" (1; #17), 6990.676578428538),
  ("funded_amnt_inv" (1; #9), 6511.295684827986),
  ("total_pymnt" (1; #35), 6302.004287074407),
  ("loan_amnt" (1; #18), 4216.866992201569),
  ("term_index" (1; #31), 2920.5626275986433),
  ("funded_amnt" (1; #8), 1379.950293531816),
  ("id" (1; #12), 1082.7244730419625),
  ("installment" (1; #15), 766.4359293325688),
  ("total_rec_int" (1; #37), 766.1573973451159),
  ("total_pymnt_inv" (1; #36), 746.2393646642449),
  ("member_id" (1; #19), 671.0634713568143),
  ("out_prncp" (1; #21), 384.9949079003527),
  ("purpose_index" (1; #25), 304.18044222623575),
  ("int_rate" (1; #16), 200.90311619125714),
  ("total_rec_late_fee" (1; #38), 161.5540843824856),
  ("annual_inc" (1; #1), 153.39556974323932),
  ("grade_index" (1; #10), 149.49854276448605),
  ("dti" (1; #6), 132.15290

In [45]:
probs = (model_1.predict(tf_test))

257/257 [==============================] - 10s 37ms/step


In [46]:
y_pred = probs.round(0)

In [47]:
len(y_pred)

256998

In [48]:
np.unique(y_pred)

array([0., 1.], dtype=float32)

In [49]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    243145
           1       0.98      0.97      0.97     13853

    accuracy                           1.00    256998
   macro avg       0.99      0.98      0.99    256998
weighted avg       1.00      1.00      1.00    256998

